# 2 Linear Equations and Computer Basics




## 2.1 Computer Arithmetic

Some knowledge of how computers perform numerical computations and how programming
languages work is useful in applied numerical work, especially if one is to
write efficient programs and avoid errors. It often comes as an unpleasant surprise to
many people to learn that exact arithmetic and computer arithmetic do not always
give the same answers, even in programs without programming errors.

Typically, computer languages such as Fortran and C allow several ways of representing
a number.

The exact details of the representation depends on the hardware but it will
suffice for our purposes to suppose that floating point numbers are stored in the form
$m2^e$, where m and e are integers with $-2^b <= m <2^b$ and $-2^d <= e < 2^d$.

The obvious way of computing this term will result in loss of precision.

These arise not only from overflow but from division by 0.

In addition,floating point numbers may get set to $NaN$, which stands for not-anumber.

Roundoff error is only one of the pitfalls in evaluating mathematical expressions.
In numerical computations, error is also introduced by the computer's inherent inability
to evaluate certain mathematical expressions exactly. For all its power, a computer
can only perform a limited set of operations in evaluating expressions. Essentially this
list includes the four arithmetic operations of addition, subtraction, multiplication
and division, as well as logical operations of comparison.

Other common functions,such as exponential, logarithmic, and trigonometric functions cannot be evaluated directly using computer arithmetic. They can only be evaluated approximately using algorithms based on the four basic arithmetic operations.

For the common functions very efficient algorithms typically exist and these are
sometimes "hardwired" into the computer's processor or coprocessor. An important
area of numerical analysis involves determining efficient approximations that can be
computed using basic arithmetic operations.



$$exp(x) = \sum^{inf}_{i=0} x^{n}/n!$$

Obviously one cannot compute the infinite sum, but one could compute a finite number
of these terms, with the hope that one will obtain sufficient accuracy for the
purpose at hand. The result, however, will always be inexact.


## 2.3 Linear Equations and the L-U Factorization

The linear equation is the most elementary problem that arises in computational
economic analysis. In a linear equation, an $n \times n$ matrix A and an n-vector b are
given, and one must compute the n-vector x that satisfies

$$A x = b$$


### Jacobi-Method


The Jacobi method is a matrix iterative method used to solve the equation Ax=b for a known square matrix A of size n×n and known vector b or length n.

$$\begin{eqnarray*}
Ax = b
\end{eqnarray*}$$

A is split into the sum of two separate matrices, D and R, such that A=D+R. Dii=Aii, but Dij=0, for i≠j. R is essentially the opposite. Rii=0, but Rij=Aij for i≠j. The solution to the equation, i.e. the value of x

, is given by the following iterative equation:

$$\begin{eqnarray*}
x^{(k+1)} = D^{-1}(b-Rx^{(k)}).
\end{eqnarray*}$$





https://www.quantstart.com/articles/Jacobi-Method-in-Python-and-NumPy







In [2]:

from numpy import array, zeros, diag, diagflat, dot

def jacobi(A,b,N=25,x=None):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # Create an initial guess if needed                                                                                                                                                            
    if x is None:
        x = zeros(len(A[0]))

    # Create a vector of the diagonal elements of A                                                                                                                                                
    # and subtract them from A                                                                                                                                                                     
    D = diag(A)
    R = A - diagflat(D)

    # Iterate for N times                                                                                                                                                                          
    for i in range(N):
        x = (b - dot(R,x)) / D
    return x

A = array([[2.0,1.0],[5.0,7.0]])
b = array([11.0,13.0])
guess = array([1.0,1.0])

sol = jacobi(A,b,N=25,x=guess)

print( "A:")
pprint(A)

print( "b:")
pprint(b)

print( "x:")
pprint(sol)

A:
array([[ 2.,  1.],
       [ 5.,  7.]])
b:
array([ 11.,  13.])
x:
array([ 7.11110202, -3.22220342])


In [4]:
#http://108.61.119.12/jacobi-method/

import numpy as np
from scipy.linalg import solve

def jacobi(A, b, x, n):

    D = np.diag(A)
    R = A - np.diagflat(D)
    
    for i in range(n):
        x = (b - np.dot(R,x))/ D
    return x



A = np.array([[4.0, -2.0, 1.0], [1.0, -3.0, 2.0], [-1.0, 2.0, 6.0]])
b = [1.0, 2.0, 3.0]
x = [1.0, 1.0, 1.0]
n = 25

x = jacobi(A, b, x, n)
print(x)

[-0.04109589 -0.28767124  0.5890411 ]


### gauss seidel method






http://austingwalters.com/gauss-seidel-method/




Using the Gauss-Seidel Method

The method is fairly straight forward, given a standard system of linear equations, Ax = b. Where, A is a matrix (often representing a series of equations), x is a vector of x variables (Gauss-Seidel method is used to solve this vector) and b is the solution vector. In Gauss-Seidel method, we then split the A matrix into Upper (U) and Lower (L) matrices (the lower matrix in this case also contains the diagonal), then iterate using the following method:



![](http://108.61.119.12/wp-content/uploads/2014/05/gauss-equation.png)


https://github.com/mmcky/nyu-econ-370/blob/master/notebooks/notes-linear-algebra.ipynb



In [ ]:
"""
ace.solvers
~~~~~~~~~~~
author:  hahnicity
https://github.com/hahnicity/ace/blob/master/ace/solvers.py
Solve a simple linear equation Ax = b in a particular way
"""
from numpy import append, array, diagonal, tril, triu
from numpy.linalg import inv, solve
from scipy.linalg import lu


def lu_decomposition(a, b):
    """
    Solve a linear equation by LU-decomposition
    Comes from LU decomposition of a matrix A s.t. A = LU
    Then
        LUx = b => Ux = y => Ly = b
    """
    _, l, u = lu(a)
    y = solve(l, b)
    return solve(u, y)


def gauss_seidel(a, b, iterations, x=None):
    """
    Solve a linear equation by the gauss seidel iteration outlined in the book
    Follows the eq:
        x = inv(L)*(b - U*x)
    """
    l = tril(a)
    upper_plus_diagonal = triu(a)
    u = upper_plus_diagonal - _diagonal_matrix(a)
    x = _check_for_initial_guess(a, x)
    for _ in xrange(iterations):
        x = inv(l).dot(b - u.dot(x))
    return x


def gauss_jacobi(a, b, iterations, x=None):
    """
    Solve a linear equation by the gauss jacobi iteration outlined in the book.
    Follows the eq:
        x = inv(D)(b - Rx)
    Where D is the diagonal matrix of A and R is the remainder s.t D + R = A
    """
    d = _diagonal_matrix(a)
    # Calculate the remainder matrix
    r = a - d
    x = _check_for_initial_guess(a, x)
    for _ in xrange(iterations):
        x = inv(d).dot(b - r.dot(x))
    return x


def _diagonal_matrix(a):
    """
    Given a square, 2D matrix a, create a diagonal matrix from it
    """
    diag = diagonal(a)
    # Create first row to initialize the correct shape
    first_row = [diag[0]] + [0 for _ in range(len(diag) - 1)]
    diag_matrix = array([first_row])
    # Construct the remaining rows in the diagonal matrix
    for index in range(1, len(diag)):
        row = [0 if index != i else diag[i] for i in range(len(diag))]
        diag_matrix = append(diag_matrix, [row], axis=0)
    return diag_matrix


def _check_for_initial_guess(a, x):
    """
    If we have not provided an initial array for x make a new one
    """
    if not x:
        x = array([1 for _ in range(a.shape[1])])
return x

## Reference


https://www3.nd.edu/~zxu2/acms40390F12/Lec-7.3.pdf

http://austingwalters.com/gauss-seidel-method/


https://stackoverflow.com/questions/17580666/improving-numpy-speed-for-gauss-seidel-jacobi-solver